<a href="https://colab.research.google.com/github/nMishelRamirez/Project_IR/blob/preprocess/ProyectoIR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Proyecto de Recuperación de la información

Integrantes:
- Ramirez Mishel
- Zaldumbide Danna

In [63]:
# Conexión con el Script
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [64]:
# Importar la librería
import sys
sys.path.append('/content/drive/MyDrive')
import gradio as gr
import numpy as np
import pandas as pd
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from rank_bm25 import BM25Okapi

In [65]:
!pip install contractions
!pip install ir_datasets
!pip install rank_bm25

In [66]:
# Enlazar el ZIP a la ruta interna donde ir_datasets lo espera
!mkdir -p ~/.ir_datasets/downloads/4e41456d7df8ee7760a7f866133bda78
!ln -s /content/drive/MyDrive/ir_datasets/cqadupstack.zip \

ln: failed to create symbolic link '\': File exists


In [67]:
import ir_datasets
dataset = ir_datasets.load("beir/cqadupstack/gaming")

# Mostrar todos los documentos del corpus
#for i, doc in enumerate(dataset.docs_iter()):
#    print(f"Document ID: {doc.doc_id}")
#    print(f"Text: {doc.text}")
#    print("-" * 80)

In [68]:
# Extraer los textos del dataset
corpus = [doc.text for doc in dataset.docs_iter()]

In [75]:
# import preprocessing as pp
from preprocessing import preprocess_dataframe

import nltk
try:
    nltk.data.find('tokenizers/punkt_tab')
except LookupError:
    nltk.download('punkt_tab')

# Crear un DataFrame
df = pd.DataFrame(corpus, columns=['raw'])

# Aplicar preprocesamiento
processed_df = preprocess_dataframe(df, lang='english', return_all=True)

In [76]:
processed_df

,raw,tokenized,normalized,filtered,stemmed,lemmatized
0,What's your Supreme Commander 2 build order. I...,"[What, is, your, Supreme, Commander, 2, build,...","[what, is, your, supreme, commander, build, or...","[supreme, commander, build, order, want, mass,...","[suprem, command, build, order, want, mass, ex...","[supreme, commander, build, order, want, mass,..."
1,"You can view how many bone shards, paintings a...","[You, can, view, how, many, bone, shards, ,, p...","[you, can, view, how, many, bone, shards, pain...","[view, many, bone, shards, paintings, runes, f...","[view, mani, bone, shard, paint, rune, found, ...","[view, many, bone, shard, painting, rune, foun..."
2,Does that mean that if no one screws up and ev...,"[Does, that, mean, that, if, no, one, screws, ...","[does, that, mean, that, if, no, one, screws, ...","[mean, one, screws, everybody, gets, daily, ra...","[mean, one, screw, everybodi, get, daili, rati...","[mean, one, screw, everybody, get, daily, rati..."
3,I have some clean rags that I found in a first...,"[I, have, some, clean, rags, that, I, found, i...","[i, have, some, clean, rags, that, i, found, i...","[clean, rags, found, first, aid, kit, would, l...","[clean, rag, found, first, aid, kit, would, li...","[clean, rag, found, first, aid, kit, would, li..."
4,I know of the extra class slots and the variou...,"[I, know, of, the, extra, class, slots, and, t...","[i, know, of, the, extra, class, slots, and, t...","[know, extra, class, slots, various, cosmetic,...","[know, extra, class, slot, variou, cosmet, stu...","[know, extra, class, slot, various, cosmetic, ..."
...,...,...,...,...,...,...
45296,I know you can't go to an 85 or 90 dungeon and...,"[I, know, you, can, not, go, to, an, 85, or, 9...","[i, know, you, can, not, go, to, an, or, dunge...","[know, go, dungeon, expect, get, rep, even, ta...","[know, go, dungeon, expect, get, rep, even, ta...","[know, go, dungeon, expect, get, rep, even, ta..."
45297,Interrogating a sectoid resulted in my earning...,"[Interrogating, a, sectoid, resulted, in, my, ...","[interrogating, a, sectoid, resulted, in, my, ...","[interrogating, sectoid, resulted, earning, re...","[interrog, sectoid, result, earn, research, cr...","[interrogating, sectoid, resulted, earning, re..."
45298,"In Assassin's Creed: Revelations, what is the ...","[In, Assassin, 's, Creed, :, Revelations, ,, w...","[in, assassin, creed, revelations, what, is, t...","[assassin, creed, revelations, highest, level,...","[assassin, creed, revel, highest, level, assas...","[assassin, creed, revelation, highest, level, ..."
45299,A friend of mine didn't realise you had to do ...,"[A, friend, of, mine, did, not, realise, you, ...","[a, friend, of, mine, did, not, realise, you, ...","[friend, mine, realise, something, order, pres...","[friend, mine, realis, someth, order, prestig,...","[friend, mine, realise, something, order, pres..."


In [71]:
# Guardar el DataFrame preprocesado
with open('/content/drive/MyDrive/ir_datasets/preprocessed_data.pkl', 'wb') as f:
    pickle.dump(processed_df[['raw', 'lemmatized']], f)


In [72]:
# Cargar el DataFrame preprocesado
with open('/content/drive/MyDrive/ir_datasets/preprocessed_data.pkl', 'rb') as f:
    loaded_df = pickle.load(f)

In [82]:
# ============================ #
#   VECTORIZACIÓN             #
# ============================ #

# TF-IDF Vectorizer
def tfidf_vectorizer(corpus):
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(corpus)
    return X, vectorizer

# BM25 Vectorizer
def bm25_vectorizer(corpus):
    tokenized_docs = [doc.split() for doc in corpus]
    # Imprimir los documentos tokenizados para ver qué está haciendo
    print("Documentos tokenizados:")
    for i, tokens in enumerate(tokenized_docs):
        print(f"Documento {i+1}: {tokens}")
    bm25 = BM25Okapi(tokenized_docs)
    return bm25

# ============================ #
#   FUNCIÓN DE BÚSQUEDA       #
# ============================ #
